In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

In [ ]:
# Załadowanie zbioru danych SQuAD z HuggingFace
dataset = load_dataset("sentence-transformers/squad")
dataset

In [ ]:
# Wykorzystanie modelu bi-enkodera: multi-qa-mpnet-base-dot-v1
# Model generuje osadzenia (embeddings) zarówno dla pytań, jak i odpowiedzi
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
model

In [ ]:
# Tworzenie osadzeń dla kolumny 'answer' (dokumenty)
# Osadzenia będą używane do porównywania z zapytaniami
answers = dataset['train']['answer']
answer_embeddings = model.encode(answers, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
# Tworzenie osadzeń dla kolumny 'question' (zapytania)
questions = dataset['train']['question']
question_embeddings = model.encode(questions, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
# Funkcja do przeprowadzenia wyszukiwania bez indeksu FAISS
def search_without_faiss(query, k=5):
    # Tworzenie osadzenia dla zapytania
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Obliczanie podobieństwa kosinusowego między zapytaniem a odpowiedziami
    scores = util.cos_sim(query_embedding, answer_embeddings)[0]
    
    # Sortowanie wyników według podobieństwa i wybór top_k wyników
    top_results = np.argsort(-scores)[:k]
    
    # Zwracanie top_k wyników
    return [(answers[idx], float(scores[idx])) for idx in top_results]

In [ ]:
# Tworzenie indeksu FAISS dla osadzeń odpowiedzi
dimension = answer_embeddings.shape[1]  # Rozmiar osadzeń
faiss_index = faiss.IndexFlatIP(dimension)  # IP oznacza Inner Product (odpowiednik kosinusowego podobieństwa)
faiss_index.add(answer_embeddings.cpu().numpy())  # Dodanie osadzeń do indeksu


In [ ]:
# Funkcja do przeprowadzenia wyszukiwania z wykorzystaniem FAISS
def search_with_faiss(query, k=5):
    # Tworzenie osadzenia dla zapytania
    query_embedding = model.encode(query, convert_to_tensor=False)
    
    # Wykorzystanie indeksu FAISS do znalezienia top_k wyników
    distances, indices = faiss_index.search(query_embedding.reshape(1, -1), k)
    
    # Zwracanie top_k wyników
    return [(answers[idx], float(distances[0][i])) for i, idx in enumerate(indices[0])]


In [ ]:
# Funkcja do oceny skuteczności wyszukiwania
def evaluate_retrieval(query, true_answer, results):
    # Sprawdzanie, czy prawdziwa odpowiedź znajduje się w wynikach top_k
    return int(any(true_answer == result[0] for result in results))

In [ ]:
# Testowanie działania na kilku przykładach
test_queries = dataset['test']['question'][:10]
test_answers = dataset['test']['answer'][:10]

# Porównanie wyników z i bez FAISS
without_faiss_scores = []
with_faiss_scores = []

for query, true_answer in zip(test_queries, test_answers):
    # Wyszukiwanie bez FAISS
    results_without_faiss = search_without_faiss(query)
    score_without_faiss = evaluate_retrieval(query, true_answer, results_without_faiss)
    without_faiss_scores.append(score_without_faiss)
    
    # Wyszukiwanie z FAISS
    results_with_faiss = search_with_faiss(query)
    score_with_faiss = evaluate_retrieval(query, true_answer, results_with_faiss)
    with_faiss_scores.append(score_with_faiss)

# Obliczanie średniej skuteczności
mean_score_without_faiss = np.mean(without_faiss_scores)
mean_score_with_faiss = np.mean(with_faiss_scores)

# Wyświetlenie wyników
print(f"Średnia skuteczność bez FAISS: {mean_score_without_faiss}")
print(f"Średnia skuteczność z FAISS: {mean_score_with_faiss}")